# **Zadanie 1(JAVA)**
# Patryk Popławski 82668
// Program wyznacza sumę i średnią kolejnych liczb z zakresu (1, ...,100) przy pomocy 10 wątków. Każdy z wątków (0-9) wylicza
//sumę i średnią dla 10 kolejnych liczb i wyświetla ją na ekranie

In [ ]:
class MyThread extends Thread {
    private int threadNum;

    public MyThread(int threadNum) {
        this.threadNum = threadNum;
    }
                //Metoda run jest wywołwana gdy wątek jest uruchamiany
    public void run() {
        int start = threadNum * 10 + 1;
        int end = start + 9;
        int sum = 0;
        int count = 0;
                //Pętla w której są wykonywane obliczenia i wywoływane dla każdej z liczb
        for (int i = start; i <= end; i++) {
            sum += i;
            count++;
        }
                //Metoda run potem wywołuje sumę oraz średnią

        System.out.println("Raczej wątek" + threadNum + "Suma =" + sum + " Średnia =" + (sum / count));
    }
}

# **Zadanie 2 (MPI)**
# Patryk Popławski 82668

//Program wyznacza sumę i średnią kolejnych liczb z zakresu (1, ...,100) przy pomocy 10 procesów. Każdy z procesów (0-9) wylicza sumę
// i średnią dla 10 kolejnych liczb wyświetla ją na ekrnaie oraz pokazuje cząstkowe wyniki do kolejnego proecesu. Ostatni 9-ty proces wylicza globlną sumę i średnią wszystkich liczb

In [ ]:
        #include <stdio.h>
        #include <mpi.h>

        #N 100 #define
        #NUMERY_PROCESOW 10 #define

                //Inicjalizacja Mpi

        int main(int argc, char* argv[]) {
        int rank, size;
        MPI_Init(&argc, &argv);
        MPI_Comm_size(MPI_COMM_WORLD, &size);
        MPI_Comm_rank(MPI_COMM_WORLD, &rank);

                //Sprawdza czy jest uruchamiany dokładnie, jeśli tak nie jest, program wyświetla printa i kończy działanie.
        if (size != NUMERY_PROCESOW) {
        printf("Program musi być uruchomiony z %d procesem.\n", NUMERY_PROCESOW);
        MPI_Abort(MPI_COMM_WORLD, 1);
        }
                //Procesy
        int start = rank * 10 + 1;
        int end = start + 9;
        int sum = 0;
        int count = 0;
        int partial_sum;
        double avg, partial_avg;

        for (int i = start; i <= end; i++) {
        sum += i;
        count++;
        }
                // Każdy proces wysyła swoje wyniki do następnego procesu ale bez ostatniego, ostatni oblicza wszyskie wyniki i oblicza globalną sumę i średnią
        partial_sum = sum;
        partial_avg = (double)sum / count;
        printf("Proces %d: Sum = %d, Average = %lf\n", rank, partial_sum, partial_avg);

        if (rank != NUMERY_PROCESOW - 1) {
        MPI_Send(&partial_sum, 1, MPI_INT, rank + 1, 0, MPI_COMM_WORLD);
        MPI_Send(&partial_avg, 1, MPI_DOUBLE, rank + 1, 0, MPI_COMM_WORLD);
        } else {
        int global_sum = 0;
        double global_avg = 0.0;

        for (int i = 0; i < NUMERY_PROCESOW - 1; i++) {
        MPI_Recv(&partial_sum, 1, MPI_INT, i, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        global_sum += partial_sum;
        MPI_Recv(&partial_avg, 1, MPI_DOUBLE, i, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        global_avg += partial_avg;
        }

        global_avg /= NUMERY_PROCESOW;
        printf("Proces %d: Globalna Sum = %d, Globalna Sredn = %lf\n", rank, global_sum, global_avg);
        }

        MPI_Finalize();
        return 0;
        }

# **Zadanie 3 (CUDA)**
# Patryk Popławski 82668


//Program wyznacza sumę i średnią kolejnych liczb z zakresu (1, ...,100) przy pomocy 10 bloków (i 10 wątków w każdym bloku)
//Każdy z bloków (0-9) wylicz sumę i średnią dla 10 kolejnych liczb (na device), które są wyśwetlane na hoście.*/

In [ ]:
#include <cuda_runtime.h>
        #include <stdio.h>

        __global__ void sum_mean_kernel(int *data, int *sums, int *counts) 
                // funkcja jest odpalana na GPU


                //Każdy blok jest wyznaczany przez wątki w danym bloku
        {
        int start = blockIdx.x * blockDim.x;
        int end = start + blockDim.x;
        int sum = 0;
        int count = 0;
        for (int i = start; i < end; i++)
        {
        sum += data[i];
        count++;
        }
        sums[blockIdx.x] = sum;
        counts[blockIdx.x] = count;
        }
                //Funkcja jest wywoływana na hoscie komputerze
        int main()
        {
        const int num_blocks = 10;
        const int block_size = 10;
        int *data, *sums, *counts;
        cudaMalloc((void**)&data, 100 * sizeof(int));
        cudaMalloc((void**)&sums, num_blocks * sizeof(int));
        cudaMalloc((void**)&counts, num_blocks * sizeof(int));
        for (int i = 0; i < 100; i++)
        {
                //Alokacja pamieci na urządzeniu GPU dla danych wejściowych, sum i liczby.

                //Wprowadzanie danych wejściowych do pamięci urządzenia GPU i wywołuje funkcję jądra
        data[i] = i + 1;
        }
        cudaMemcpy(data, data, 100 * sizeof(int), cudaMemcpyHostToDevice);
        sum_mean_kernel<<<num_blocks, block_size>>>(data, sums, counts);
        cudaMemcpy(sums, sums, num_blocks * sizeof(int), cudaMemcpyDeviceToHost);
        cudaMemcpy(counts, counts, num_blocks * sizeof(int), cudaMemcpyDeviceToHost);
        for (int i = 0; i < num_blocks; i++)

                //Wracanie do hosta po obliczeniach i wyświetlanie sumy oraz średniej z każdego bloku na device
        {
        int start = i * block_size + 1;
        int end = start + block_size - 1;
        int sum = sums[i];
        int count = counts[i];
        float mean = (float)sum / count;
        printf("Block %d: Sum = %d, Mean = %f\n", i, sum, mean);
        }
        cudaFree(data);
        cudaFree(sums);
        cudaFree(counts);
        return 0;